<a href="https://colab.research.google.com/github/ghommidhWassim/GNN-variants/blob/main/graphSaint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!python -c "import torch; print(torch.__version__)"
!python -c "import torch; print(torch.version.cuda)"
!pip install torchvision
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.6.0+cu124.html

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
2.6.0+cu124
12.4
Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html


In [2]:
# !pip install torch torchvision torchaudio --quiet
# !pip install scipy numpy --quiet
# !git clone https://github.com/graphsaint/graphsaint.git  # if you want to use official repo, or upload your own files

import os
import numpy as np
import scipy.sparse as sp
import torch
import time
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from torch.nn import Linear
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.utils import to_scipy_sparse_matrix
from torch_geometric.loader import GraphSAINTNodeSampler, GraphSAINTEdgeSampler, GraphSAINTRandomWalkSampler

# (You will have to upload or place the GraphSAINT modules or install them if available)
# For simplicity, assume graphsaint package is already in your environment or uploaded as files.


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ------------------- Load Dataset -------------------
def dataset_load():
    print(f"Using device: {device}")
    dataset = Planetoid(root='data/Planetoid', name='PubMed', transform=NormalizeFeatures())
    data = dataset[0].to(device)
    return dataset.num_features, data, dataset.num_classes

num_features, data, num_classes = dataset_load()


Using device: cuda


In [4]:
data = data.cpu()
loader_SAINT_256_node = GraphSAINTNodeSampler(data, batch_size=500, num_steps=4, sample_coverage=10)
#loader_SAINT_256_edge = GraphSAINTEdgeSampler(data, batch_size=500, num_steps=4, sample_coverage=10)
#loader_SAINT_256_RW = GraphSAINTRandomWalkSampler(data, batch_size=500, walk_length=2, num_steps=4, sample_coverage=10)


Compute GraphSAINT normalization: : 198778it [00:00, 927460.84it/s]                          
Compute GraphSAINT normalization: : 198592it [03:03, 1085.04it/s]                          
Compute GraphSAINT normalization: : 197445it [00:00, 2301321.51it/s]        


In [5]:
class GCN(torch.nn.Module):
  def __init__(self, hidden_channels):
    super().__init__()
    self.conv1 = GCNConv(num_features, hidden_channels)
    self.conv2 = GCNConv(hidden_channels, num_classes)

  def forward(self, x, edge_index, edge_weight=None):
      x = self.conv1(x, edge_index, edge_weight)
      x = x.relu()
      x = F.dropout(x, p=0.5, training=self.training)
      x = self.conv2(x, edge_index, edge_weight)
      return x


In [6]:
model = GCN(hidden_channels=16).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss().to(device)

In [7]:
def train(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for batch in loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index, batch.edge_norm)
        loss = criterion(out, batch.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# ------------------- Evaluation Function -------------------
@torch.no_grad()
def test(model, full_data):
    model.eval()
    out = model(full_data.x.to(device), full_data.edge_index.to(device))
    pred = out.argmax(dim=1)
    correct = (pred[full_data.test_mask] == full_data.y[full_data.test_mask].to(device)).sum()
    acc = int(correct) / int(full_data.test_mask.sum())
    return acc

# ------------------- Run Training -------------------
def run(loader, method_name):
    # Use the global num_classes variable instead of trying to access it from the data object


    for epoch in range(1, 101):
        loss = train(model, loader, optimizer, criterion)
        print(f'{method_name} | Epoch {epoch:03d}, Loss: {loss:.4f}')

    acc = test(model, data)
    print(f'{method_name} | Final Test Accuracy: {acc:.4f}')

# ------------------- Execute Training for Each Loader -------------------
run(loader_SAINT_256_node, "GraphSAINT-NodeSampler")
#run(loader_SAINT_256_edge, "GraphSAINT-EdgeSampler")
#run(loader_SAINT_256_RW, "GraphSAINT-RandomWalkSampler")


GraphSAINT-NodeSampler | Epoch 001, Loss: 1.0787
GraphSAINT-NodeSampler | Epoch 002, Loss: 1.0382
GraphSAINT-NodeSampler | Epoch 003, Loss: 0.9872
GraphSAINT-NodeSampler | Epoch 004, Loss: 0.9641
GraphSAINT-NodeSampler | Epoch 005, Loss: 0.9311
GraphSAINT-NodeSampler | Epoch 006, Loss: 0.8944
GraphSAINT-NodeSampler | Epoch 007, Loss: 0.8861
GraphSAINT-NodeSampler | Epoch 008, Loss: 0.8129
GraphSAINT-NodeSampler | Epoch 009, Loss: 0.8066
GraphSAINT-NodeSampler | Epoch 010, Loss: 0.7673
GraphSAINT-NodeSampler | Epoch 011, Loss: 0.7819
GraphSAINT-NodeSampler | Epoch 012, Loss: 0.7112
GraphSAINT-NodeSampler | Epoch 013, Loss: 0.7072
GraphSAINT-NodeSampler | Epoch 014, Loss: 0.6696
GraphSAINT-NodeSampler | Epoch 015, Loss: 0.6554
GraphSAINT-NodeSampler | Epoch 016, Loss: 0.6377
GraphSAINT-NodeSampler | Epoch 017, Loss: 0.6181
GraphSAINT-NodeSampler | Epoch 018, Loss: 0.5968
GraphSAINT-NodeSampler | Epoch 019, Loss: 0.6515
GraphSAINT-NodeSampler | Epoch 020, Loss: 0.5709
GraphSAINT-NodeSampl

In [8]:
!git clone https://github.com/anderskm/gputil.git

Cloning into 'gputil'...
remote: Enumerating objects: 456, done.
remote: Total 456 (delta 0), reused 0 (delta 0), pack-reused 456 (from 1)
Receiving objects: 100% (456/456), 83.68 KiB | 11.95 MiB/s, done.
Resolving deltas: 100% (263/263), done.


In [9]:
pip install gputil

  Preparing metadata (setup.py) ... done
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7392 sha256=9603a18d140bc2e9b368d954f7252290f84ee7eeba5f271547fa5c2bd54ccb15
  Stored in directory: /root/.cache/pip/wheels/2b/4d/8f/55fb4f7b9b591891e8d3f72977c4ec6c7763b39c19f0861595
Successfully built gputil


In [10]:
import GPUtil
GPUtil.showUtilization()
GPUs = GPUtil.getGPUs()
GPUs[0].memoryUsed

| ID | GPU | MEM |
------------------
|  0 |  0% |  1% |


218.0

In [11]:
print(f"GPU memory allocated: {torch.cuda.memory_allocated()/1024**2:.2f} MB")
print(f"Max GPU memory used:  {torch.cuda.max_memory_allocated()/1024**2:.2f} MB")

GPU memory allocated: 16.38 MB
Max GPU memory used:  73.44 MB


In [12]:
print(f"Total reserved memory: {torch.cuda.memory_reserved() / 1024**2:.2f} MB")
print(f"Max reserved memory:   {torch.cuda.max_memory_reserved() / 1024**2:.2f} MB")


Total reserved memory: 82.00 MB
Max reserved memory:   82.00 MB
